In [1]:
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from pprint import pprint
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

from miniai.training import *

In [2]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
from fastcore.test import test_close

c:\Users\grzeg\Miniconda3\envs\fastai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'
     
logging.disable(logging.WARNING)

# Hugging Face Datasets

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.

load_dataset_builder is used to insect the data before loading it to your filesystem

In [4]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)

In [5]:
pprint(ds_builder.info.features)

{'image': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['T - shirt / top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle boot'],
                     id=None)}


In [6]:
pprint(ds_builder.info.splits)

{'test': SplitInfo(name='test',
                   num_bytes=5235160,
                   num_examples=10000,
                   shard_lengths=None,
                   dataset_name='fashion_mnist'),
 'train': SplitInfo(name='train',
                    num_bytes=31304707,
                    num_examples=60000,
                    shard_lengths=None,
                    dataset_name='fashion_mnist')}


# Loading DataSet Dict from HF

In [7]:
dsd = load_dataset(name)
dsd

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [8]:
train, test = dsd["train"], dsd["test"]

In [9]:
train

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [10]:
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [11]:
x,y = ds_builder.info.features
x,y

('image', 'label')

In [12]:
img = train[0][x]
img

In [13]:
xb = train[:5][x]
yb = train[:5][y]

xb, yb

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 [9, 0, 0, 3, 0])

we want to now know which IDs map to their corresponding labels 

In [14]:
featy = train.features[y]
pprint(featy)

ClassLabel(names=['T - shirt / top',
                  'Trouser',
                  'Pullover',
                  'Dress',
                  'Coat',
                  'Sandal',
                  'Shirt',
                  'Sneaker',
                  'Bag',
                  'Ankle boot'],
           id=None)


In [15]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [26]:
# Note that we can do all of our work in the collation function
# 1. go through each item in a batch
# 2. grab x from each item
# 3. convert it to a tensor
# 4. stack them together into one tensor

def collate_fn(b):
    return {x:torch.stack([torch.flatten(TF.to_tensor(o[x])) for o in b]),
            y:tensor([o[y] for o in b])}

In [27]:
bs = 16

dl = DataLoader(train, collate_fn=collate_fn, batch_size=bs)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([16, 784]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

The second approach is to use with_transform method that HF datasets have and it allows us to transform a batch of data. Note that we need to return a new transformed batch

In [28]:
def transforms(b):
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]
    return b

DataLoader from pytorch can taka DataSet Dictionary 

In [40]:
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([16, 784]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [30]:
def _transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

transformi = inplace(_transformi)

In [31]:
r = train.with_transform(transformi)[0]
r[x].shape,r[y]

(torch.Size([784]), 9)

In [32]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [33]:
t = torch.tensor([[[1, 2],
                   [3, 4]],
                  [[5, 6],
                   [7, 8]]])

t.shape

torch.Size([2, 2, 2])

In [34]:
torch.flatten(t)

tensor([1, 2, 3, 4, 5, 6, 7, 8])

In [35]:
torch.flatten(t, start_dim=1)

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]])

In [81]:
train

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [54]:
tdsf = train.with_transform(transformi)
tdsf

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [57]:
r = tdsf[0]
r[x].shape, r[y]

(torch.Size([784]), 9)

In [58]:
dl = DataLoader(tdsf, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y]

(torch.Size([16, 784]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [75]:
b

{'image': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'label': tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9])}

we note that our dataloader returns now a dictionary for each batch and before we had a tuple. In order to have tuples we can use itemgetter and replace default collate function in DataSet

## itemgetter

itemgetter allows us to get values with a specific key or attribute name from python objects

In [61]:
d = dict(a=1,b=2,c=3)
ig = itemgetter('a','c')
ig(d)

(1, 3)

In [62]:
class D:
    def __getitem__(self, k): return 1 if k=='a' else 2 if k=='b' else 3

d = D()
ig(d)

(1, 3)

In [69]:
batch = dict(a=[1],b=[2]), dict(a=[3],b=[4]), dict(a=[5],b=[6])
batch

({'a': [1], 'b': [2]}, {'a': [3], 'b': [4]}, {'a': [5], 'b': [6]})

default_collate function from pytorch will grab all all items from an iterator with the same key name and stack them together. We need to ensure that all elements in the iterator have the same keys

In [71]:
default_collate(batch)

{'a': [tensor([1, 3, 5])], 'b': [tensor([2, 4, 6])]}

In [72]:
list(tdsf.features)

['image', 'label']

In [73]:
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [78]:
b

{'image': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'label': tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9])}

In [77]:
get = itemgetter(*tdsf.features)
get(b)

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

In [87]:
dl = DataLoader(tdsf, batch_size=4)
b = next(iter(dl))
b

{'image': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'label': tensor([9, 0, 0, 3])}

In [88]:
dlf = DataLoader(tdsf, batch_size=4, collate_fn=collate_dict(tdsf))
b = next(iter(dlf))
b

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([9, 0, 0, 3]))

huggingface use to do things with dictionaries and other libraries usually work with tuples 